# Azure Evaluation On Audio Length

1. Compute Audio Length for Each Transcription
2. Compute Error
3. Generate Plots
***
### Error Metrics
1. WER
2. ROUGE

***

## Using Custom Kernel on SCC

SCC sometimes has the problem with installed library not importable [`module not found` error], this is an alternative.

Assuming you have a conda environment created, you would do the following:
1. `conda install -c anaconda ipykernel` 
2. `python -m ipykernel install --user --name=<env name>`
3. If the new kernel cannot be found, relaunch a new SCC instance

**Remember to switch to the conda env kernel**

In [ ]:
ds_transcript_path = "/projectnb/ds549/projects/AImpower/datasets/updated_annotation_deid_full"

In [ ]:
!pip install pandas numpy scipy tqdm

***

# Imports & Ingestion of Data
**We will be using the data generated from `azure-stu-eval.ipynb`.**

In [ ]:
import pandas as pd
import numpy as np
import scipy 
import os
import sys
import re
from tqdm import tqdm
import matplotlib.pyplot as plt

In [ ]:
net_data = pd.DataFrame(columns=["Filename", "Start_time", "End_time", "Transcript"]) 

cols = list(pd.read_csv("net_aigenerated_data_azure_performance_stu.csv", nrows=1))
print(cols)

net_aigenerated_data_azure = pd.read_csv('/projectnb/ds549/projects/AImpower/evaluation-azure/net_aigenerated_data_azure_performance_stu.csv', delimiter=',', usecols =[i for i in cols if "Unnamed:" not in i])

In [ ]:
for folder in os.listdir(ds_transcript_path):
    if folder == "command_stats.xlsx" or folder == "command_stats.csv":
        continue
    for audio_sample in os.listdir(os.path.join(ds_transcript_path, f"{folder}")):
        if ("_A.txt" in audio_sample):
            net_data = pd.concat([net_data, pd.read_csv(os.path.join(ds_transcript_path, f"{folder}/{audio_sample}"), sep="\t", names=["Start_time", "End_time", "Transcript"]).assign(Filename=f"D{folder}_A")])
        if ("_B.txt" in audio_sample):
            net_data = pd.concat([net_data, pd.read_csv(os.path.join(ds_transcript_path, f"{folder}/{audio_sample}"), sep="\t", names=["Start_time", "End_time", "Transcript"]).assign(Filename=f"D{folder}_B")])
        if ("P" in audio_sample):
            net_data = pd.concat([net_data, pd.read_csv(os.path.join(ds_transcript_path, f"{folder}/{audio_sample}"), sep="\t", names=["Start_time", "End_time", "Transcript"]).assign(Filename=f"P{folder}")])

In [ ]:
mask_pattern = r"\<.*?\>"
repetition_pattern = r"\[.*?\]"
annotation_pattern = r"/\w"


net_data = net_data.assign(Cleaned_Transcript=net_data['Transcript'].apply(lambda x: re.sub(annotation_pattern, "", re.sub(repetition_pattern, "", re.sub(mask_pattern, "", x)))))
net_data = net_data.assign(Stutterance_Count=net_data['Transcript'].apply(lambda x: len(re.findall(mask_pattern, x)) + len(re.findall(repetition_pattern, x)) + len(re.findall(annotation_pattern, x))))

In [ ]:
net_data

In [ ]:
## Assign back the cleaned transcript and original transcript
merged_data = net_aigenerated_data_azure.merge(
    net_data[['Filename', 'Start_time', 'Cleaned_Transcript', 'Transcript']],
    on=['Filename', 'Start_time'],
    how='left'
)

net_aigenerated_data_azure['Cleaned_Transcript'] = merged_data['Cleaned_Transcript']
net_aigenerated_data_azure['GroundTruth_Transcript'] = merged_data['Transcript']

In [ ]:
net_aigenerated_data_azure

**Now we have raw data of all audio transcriptions from datasets [updated_annotation_deid_full] in ```net_data``` and AI predicted transcriptions in ```net_aigenerated_data_azure```**

In [ ]:
## SAVING

net_aigenerated_data_azure.to_csv('net_aigenerated_data_azure_performance_stu.csv', sep=',')

***

## Compute Audio Length

In [ ]:
net_aigenerated_data_azure = net_aigenerated_data_azure.assign(Duration=net_aigenerated_data_azure['End_time']-net_aigenerated_data_azure['Start_time'])

***

## Visualization of Scores

In [ ]:
plt.figure(figsize=(6, 4))
plt.scatter(
    net_aigenerated_data_azure["Duration"], 
    net_aigenerated_data_azure["WER"], 
    alpha=0.7  # Handle overlapping points
)

plt.title("WER vs Audio Duration", fontsize=16)
plt.xlabel("Audio Duration", fontsize=14)
plt.ylabel("WER", fontsize=14)
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(6, 4))
plt.scatter(
    net_aigenerated_data_azure["Duration"], 
    net_aigenerated_data_azure["rouge1-precision"], 
    facecolors="none", edgecolors='r',
    label="Precision",
    marker="8",
    alpha=0.7  # Handle overlapping points
)

plt.scatter(
    net_aigenerated_data_azure["Duration"], 
    net_aigenerated_data_azure["rouge1-recall"], 
    facecolors="none", edgecolors='g',
    label="Recall",
    marker="^",
    alpha=0.7  # Handle overlapping points
)

plt.scatter(
    net_aigenerated_data_azure["Duration"], 
    net_aigenerated_data_azure["rouge1-f1"], 
    facecolors="none", edgecolors='b',
    label="F1",
    marker=".",
    alpha=0.7  # Handle overlapping points
)

plt.title("Rouge-1 vs Audio Duration", fontsize=16)
plt.xlabel("Audio Duration", fontsize=14)
plt.ylabel("Rouge Score", fontsize=14)
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(6, 4))
plt.scatter(
    net_aigenerated_data_azure["Duration"], 
    net_aigenerated_data_azure["rouge2-precision"], 
    facecolors="none", edgecolors='r',
    label="Precision",
    marker="8",
    alpha=0.7  # Handle overlapping points
)

plt.scatter(
    net_aigenerated_data_azure["Duration"], 
    net_aigenerated_data_azure["rouge2-recall"], 
    facecolors="none", edgecolors='g',
    label="Recall",
    marker="^",
    alpha=0.7  # Handle overlapping points
)

plt.scatter(
    net_aigenerated_data_azure["Duration"], 
    net_aigenerated_data_azure["rouge2-f1"], 
    facecolors="none", edgecolors='b',
    label="F1",
    marker=".",
    alpha=0.7  # Handle overlapping points
)

plt.title("Rouge-2 vs Audio Duration", fontsize=16)
plt.xlabel("Audio Duration", fontsize=14)
plt.ylabel("Rouge Score", fontsize=14)
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(6, 4))
plt.scatter(
    net_aigenerated_data_azure["Duration"], 
    net_aigenerated_data_azure["rougel-precision"], 
    facecolors="none", edgecolors='r',
    label="Precision",
    marker="8",
    alpha=0.7  # Handle overlapping points
)

plt.scatter(
    net_aigenerated_data_azure["Duration"], 
    net_aigenerated_data_azure["rougel-recall"], 
    facecolors="none", edgecolors='g',
    label="Recall",
    marker="^",
    alpha=0.7  # Handle overlapping points
)

plt.scatter(
    net_aigenerated_data_azure["Duration"], 
    net_aigenerated_data_azure["rougel-f1"], 
    facecolors="none", edgecolors='b',
    label="F1",
    marker=".",
    alpha=0.7  # Handle overlapping points
)

plt.title("Rouge-L vs Audio Duration", fontsize=16)
plt.xlabel("Audio Duration", fontsize=14)
plt.ylabel("Rouge Score", fontsize=14)
plt.grid(True)
plt.legend()
plt.show()